### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import VideoPlayer, UnindexedStream
import os
import pandas as pd

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
#gateway = '10.246.27.131:54672'
#gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
#stream = 'object-detector-input-video'
stream = 'object-detector-output-video'
#stream = 'generated-video'
#stream = 'object-detector-input-grid-video'

In [ ]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
unindexed_stream = UnindexedStream(pravega_client, scope=scope, stream=stream)
stream_info = unindexed_stream.get_stream_info()
last_frame_number = None
for frame in unindexed_stream.read_video_frames(from_stream_cut=stream_info.head_stream_cut, cameras=[0])[0]:
    header = 'camera=%02d, frameNumber=%08d, timestamp=%s' % (frame['camera'], frame['frameNumber'], pd.to_datetime(frame['timestamp'], unit='ms', utc=True))
    print(header, end='\r')
    if last_frame_number is not None:
        expected = last_frame_number + 1
        if frame['frameNumber'] != expected:
            print('%s: Unexpected frame number: expected=%08d' % (header, expected))
    last_frame_number = frame['frameNumber']